In [ ]:
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',google_api_key=os.getenv('GEMINI_API_KEY'))
# print(llm.invoke('my name is prem '))

content="Hi Prem! It's nice to meet you. How can I help you today?" additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b8d09-9515-7a93-8375-0afa27451df5-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 6, 'output_tokens': 18, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}


In [ ]:
"""
pseudocode :
result=usrdata | llm | stroutparser ---> 3 chains 
1. meal_plan_chain = result | llm |strparser
2. workout_plan_chain = result | llm | strparser
3. weekly_stratergy_chain = mean_plan_chain & workout_plan_chian | llm | strparser

resp = RunnableParallel(meal plan , workout plan , weekly stratergy plan )

"""

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# name = input('Enter your name :')
age =int(input('Enter your age :'))
weight_kg =int(input('Enter your weight in kgs :'))
height_cm=int(input('Enter your height in (cm) :'))
activity_level=input('Enter your activity level between High/moderate/low :')
diet_preference =input('Enter the diet preference veg/nonveg :')
fitness_goal=input('Enter your fitness goal :')

user_profile = {
#   "name":name,
  "age":age,
  "weight_kg":weight_kg,
  "height_cm":height_cm,
  "activity_level": activity_level,
  "diet_preference":diet_preference,
  "fitness_goal": fitness_goal
}

# llm invoke 
llm = ChatGoogleGenerativeAI(model ='gemini-2.5-flash-lite',google_api_key=os.getenv('GEMINI_API_KEY'))
prompt = PromptTemplate(template='This is my user data {user_data}',input_variables=['user_data'])
base_chain = prompt | llm |StrOutputParser()

# meal chain 
meal_plan_prompt = PromptTemplate(template='Based on this usr details {profile_text} give met the full meal plan to achieve my target Goal ',input_variables=['profile_text'])
meal_chain = meal_plan_prompt | llm | StrOutputParser()

# workout_plan_chain
workout_plan_prompt = PromptTemplate(template='Based on this usr details {profile_text} give met the full Workout plan to achieve my target Goal ',input_variables=['profile_text'])
workout_chain = workout_plan_prompt | llm | StrOutputParser()

#weekly_stratergy_chain


from langchain_core.runnables import RunnableParallel
final_chain=(
 base_chain |  RunnableParallel(
    {
        'meal_plan':meal_chain,
        'workout_plan':workout_chain,
    }
)
)

result = final_chain.invoke({'user_data':user_profile})
print(result)

weekly_stratergy_prompt =PromptTemplate(template='Based on {output} give me a weakly plan to achive my target for 6 months.',input_variables=['output'])
weekly_stratergy_chain = weekly_stratergy_prompt | llm | StrOutputParser()

print(weekly_stratergy_chain.invoke({'output':result}))




{'meal_plan': 'It\'s fantastic that you\'ve provided such comprehensive details about your user! This is a great starting point. However, to create a *full and truly effective meal plan* to achieve their "fit" goal, we need to bridge a few gaps.\n\nThe current data allows us to make *informed suggestions*, but a precise meal plan requires more specificity on their definition of "fit" and their current routine.\n\n**Here\'s why we can\'t give a *full meal plan* yet, and what we need to do first:**\n\n**1. Defining "Fit":**\n\n*   **What does "fit" mean *to them*?** This is the most crucial missing piece. Does it mean:\n    *   **Muscle Gain:** Higher protein, sufficient carbs for energy, and a slight caloric surplus.\n    *   **Fat Loss:** Caloric deficit, balanced macros with emphasis on protein and fiber for satiety.\n    *   **Improved Endurance:** Higher carbohydrate intake for sustained energy, adequate protein for muscle repair.\n    *   **General Health & Well-being:** A balanced